In [67]:
import pandas as pd
import numpy as np
import math
import random
import sys
import bisect
import random
import plotly

# Get libraries for buttons futture project
import ipywidgets as widgets
from IPython.display import display

In [66]:
!pip install plotly


   ---------------------------------------- 14.8/14.8 MB 61.9 MB/s eta 0:00:00


# Classes

## Deck Class

In [2]:
class Deck:
    
    CARD_VALUES = {
        '2': 2, '3': 3, '4': 4, '5': 5, '6': 6,
        '7': 7, '8': 8, '9': 9, '10': 10,
        'J': 10, 'Q': 10, 'K': 10 , 'A': 11  # Start Aces as 11
    }
    
    COUNT_VALUES = {
        '2': 1, '3': 1, '4': 1, '5': 1, '6': 1,
        '7': 0, '8': 0, '9': 0, 
        '10': -1, 'J': -1, 'Q': -1, 'K': -1 , 'A': -1
    }
    
    INVERSE_COUNT_VALUES = {
        '2': -1, '3': -1, '4': -1, '5': -1, '6': -1,
        '7': 0, '8': 0, '9': 0, 
        '10': 1, 'J': 1, 'Q': 1, 'K': 1 , 'A': 1
    }
    
    # Testing
    # CARD_VALUES = {
    #     # '10': 10, 'J': 10, 'Q': 10, 'K': 10
    #     'A': 11 , 'J': 10, '9': 9
    # }
    
    def __init__(self, num_decks=1):
        """Initialize the deck with the given number of decks"""
        # random.seed(25)
        self.num_decks = num_decks
        self.cards = self._generate_deck() * num_decks
        self.initial_deck_size = len(self.cards)
        self.shuffle()
        
    def _generate_deck(self):
        """Generate a single deck of cards"""
        suits = ['♠', '♥', '♦', '♣']
        values = list(self.CARD_VALUES.keys())
        return [(value, suit) for value in values for suit in suits]
    
    def shuffle(self):
        """Shuffle the deck."""
        random.shuffle(self.cards)
        
    def get_count(self):
        """Return the running count (Will use inverse count values as we are counting based on whats remaining in the deck)"""
        # Get the ranks of all cards left in the deck
        deck_ranks = list(map(lambda x: x[0], d.cards))
        
        # Map and sum the ranks to their inverse counts
        count = sum(map(self.INVERSE_COUNT_VALUES.get, deck_ranks))
        
        return count
        
    def insert_cut(self, deck_size, cut_idx):
        """Set index of cut card placement"""
        index = int(deck_size * cut_idx)
        self.cards.insert(index, 'CUT')
        
    def deal_card(self):
        """Deal a single card from the deck."""
        return self.cards.pop() if self.cards else None
    
    def pull_card(self, target_value):
        """finds a target value out of the deck and deals it, returns None if not found"""
        index = next((i for i, (r, s) in enumerate(self.cards) if self.CARD_VALUES[r] == target_value), None)
        return self.cards.pop(index) if index is not None else None
    
    def insert_cut_card(self, depth=0.8):
        pass
    
    def check_deck(self):
        return len(self.cards)
    
    def print_deck(self):
        print(self.cards)
    
        

In [133]:
# Deck testing
d = Deck()
card_list = []
card_list.append(d.deal_card())
card_list

[('2', '♥')]

In [134]:
card_list.append(d.pull_card(9))
card_list

[('2', '♥'), ('9', '♥')]

In [135]:
d.check_deck()

50

In [136]:
d.get_count()

1

## Chip Class

In [3]:
class PlayerChips:
    
    def __init__(self, num_players, unit=1, buy_in=0):
        """Initialize chips for players"""
        self.bankroll = buy_in
        self.unit = unit
        
        self.win_X = 2
        self.blackjack_X = 2.5 
        self.insurance_X = 3
        
    def place_bet(self, bet_size):
        """Place bet (Will allow negative chips to calculate EV)"""
        self.bet = bet_size
        self.bankroll -= bet_size
        
    def win_bet(self, multiplier=2):
        winnings = self.bet * multiplier
        self.bankroll += winnings
        self.bet = 0
        return winnigns
    
    def lose_bet(self):
        self.bet = 0
    
    def push_bet(self):
        self.bankroll += self.bet
        self.bet = 0
        
    def double_bet(self):
        pass
    
    def get_chip_count(self):
        """Get the chip count"""
        return self.bankroll
        
    
        
    

## Blackjack Class

In [17]:
class Blackjack:
    
    def __init__(self, num_players=1, num_decks=1, HIT_SOFT_17 = False, cut=.8):
        """Initialize game parameters"""
        self.deck = Deck(num_decks)
        self.num_players = min(num_players, 7) # Set max players at table to 7
        self.players_hands = {i :[[]] for i in range(1, num_players+1)} # set players with empty hands (double indexed for multiple hands)
        self.dealer_hand = []
        self.num_decks = num_decks
        self.INITIAL_HAND_SIZE = 2
        
        self.cut = 1-cut
        self.cut_reached = False
        self.reset_sim = False
        self.FULL_DECK = len(self.deck.cards)
        
        self.HIT_SOFT_17 = HIT_SOFT_17
        self.MAX_SPLITS = 3
           
        # Define buttons
        # self.hit_button = widgets.Button(description="Hit")
        # self.stand_button = widgets.Button(description="Stand")
        # self.split_button = widgets.Button(description="Split")
        # self.double_button = widgets.Button(description="Double")
        
    def insert_cut(self):
        """Set index of cut card placement"""
        self.deck.insert_cut(self, self.FULL_DECK, self.cut)
        
    def auto_shuffle(self):
        """Add all cards back to deck and re-shuffle (basically just create a new deck)"""
        self.deck = Deck(num_decks)
        
    def get_count(self):
        return self.deck.get_count()
        
    def initial_deal(self):
        """Deal two cards to each player in sequence ending with dealer"""
        for _ in range(self.INITIAL_HAND_SIZE):
            for player, hand in self.players_hands.items():
                # If cut card reached draw another card for player and set flag to re-shuffle
                card = self.deck.deal_card()
                if card == 'CUT':
                    self.cut_reached = True
                    card = self.deck.deal_card()
                hand[0].append(card)
            
            card = self.deck.deal_card()
            # If cut card reached draw another card for dealer and set flag to re-shuffle
            if card == 'CUT':
                self.cut_reached = True
                card = self.deck.deal_card()        
            self.dealer_hand.append(card)
    
    def rigged_deal(self, total=0, upcard=6, value1=11, value2=10):
        """Rigs the deal for player 1 while dealing randomly to others. Dealers up card is also pre set"""
        
        v1 = value1
        v2 = value2
        
        if 4 <= total <= 20:
            # if total is set between 4 and 20 (no aces for soft values), generate random cards to equal that total
            v1 = random.randint(max(total-10, 2), min(10, total-2))
            v2 = total - v1
        
        # Define rigged hands for Player 1 and Dealer Upcard
        rigged_cards = [v1, v2]
            
        for i in range(self.INITIAL_HAND_SIZE):
            for player, hand in self.players_hands.items():
                # Rig the deal for player 1
                if player == 1:
                    card = self.deck.pull_card(rigged_cards[i]) 
                    # Check if card exists in the deck set flag to initiate new shoe if true
                    if card is None:
                        self.reset_sim = True
                else: 
                    card = self.deck.deal_card()
                    # Check if cut card has been reached on other players
                    if card == 'CUT':
                        self.cut_reached = True
                        card = self.deck.deal_card()
                    
                hand[0].append(card)

            dealer_card = self.deck.pull_card(upcard) if i == 0 else self.deck.deal_card()
            self.dealer_hand.append(dealer_card)
            
    def check_lucky_lucky(self, hand):
        """ 19 or 20 = 2:1 -> Implied Odds 33%
            21 = 3:1 -> Implied Odds 25%
            Suited 21 = 15:1 -> Implied Odds 6.25%
            6 7 8 = 30:1 => Implied Odds 3.3%
            7 7 7 = 50:1 -> Implied Odds 2%
            Suited 6 7 8 = 100:1 -> Implied Odds 1%
            Suited 7 7 7 = 200:1 -> Implied Odds 0.5%"""
        
        c1 = self.dealer_hand[0]
        c2 = hand[0]
        c3 = hand[1]
        
        s1 = self.get_suit(c1)
        s2 = self.get_suit(c2)
        s3 = self.get_suit(c3)
        
        v1 = self.get_value(c1)
        v2 = self.get_value(c2)
        v3 = self.get_value(c3)
        
        total = v1 + v2 + v3
        
        # 7s
        if v1 == v2 == v3 == 7:
            if s1 == s2 == s3:
                return 'S777'
            else:
                return 'U777'
        # 6,7,8
        elif set((6,7,8)) == set((6,7,8)):
            if s1 == s2 == s3:
                return 'S678'
            else:
                return 'U678'
        
        # 21 total
        elif total == 21:
            if s1 == s2 == s3:
                return 'ST21'
            else:
                return 'UT21'
        
        # 19 or 20 total
        elif total == 20
            return 'T20'
        elif total == 19:
            return 'T19'
        
        # No lucky
        else:
            return ''
        
    def calculate_hand_value(self, hand):
        """Calculate the hands total with adjustment for Ace"""
        total = 0
        aces = 0
        soft = False
        
        for rank, suit in hand:
            total += self.deck.CARD_VALUES[rank]
            if rank == 'A':
                aces += 1
            
        while total > 21 and aces > 0:
            # Set ace value to 1 and remove 11 ace value 
            aces -= 1
            total -= 10

        soft = aces >= 1 and total < 21
            
        return total, soft
    
    def get_player_action(self, hand, split_counter, aces_split):
        """Take player input of either hit, stand, double, or split"""
        can_double = (len(hand) == 2)
        can_split = (len(hand) == 2 and self.deck.CARD_VALUES[hand[0][0]] == self.deck.CARD_VALUES[hand[1][0]]) # Check that there are only 2 cards and they have the same numeric value
        
        # Check split aces, either split again or force stand
        if aces_split:
            # Check if dealt card is ace (mini selection pool)
            print(f'Card 1: {hand[0]}')
            print(f'Card 2: {hand[1]}')
            if self.get_rank(hand[1]) == 'A' and split_counter < self.MAX_SPLITS:
                while True:
                    options = "[S]tand, S[p]lit"
                    valid_choices = ['s','p','q']
                    choice = input(f'Chose: {options}: ').strip().lower()
                    if choice in valid_choices:
                        return choice
                    else:
                        print('Invalid choice. Please try again.')
            else:
                print('Force Stand.')
                return 's'
            
        # List options and take input
        options = "[H]it, [S]tand"
        if can_split and split_counter < self.MAX_SPLITS:
            options += ", S[p]lit"
        if can_double:
            options += ", [D]ouble Down"
        
        while True:
            valid_choices = ['q','h','s']
            if can_split and split_counter < self.MAX_SPLITS:
                valid_choices.append('p')
            if can_double:
                valid_choices.append('d')
                
            choice = input(f'Chose: {options}: ').strip().lower()
            
            if choice in valid_choices:
                return choice
            else:
                print('Invalid choice. Please try again.')
    
    def hit(self, hand):
        """Add a card to the target hand"""
        card = self.deck.deal_card()
        # If cut card is reached, set flag to true and deal another card to hand
        if card == 'CUT':
            self.cut_reached = True
            card = self.deck.deal_card()
        hand.append(card)
    
    def split(self, hands, hand):
        """split cards into 2 two hands"""
        # Add an empyt hand to the end of the players hadns
        hands.append([])
        hand_count = len(hands)
        
        # Pop card from current hand into the last hand
        hands[hand_count-1].append(hand.pop())
        
    def check_bust(self, hand):
        """check to see if hand is busted (over 21)"""
        total, soft = self.calculate_hand_value(hand)
        return True if total > 21 else False
    
    def get_rank(self, card):
        return card[0]
    
    def get_suit(self, card):
        return card[1]
    
    def get_value(self, card):
        return self.deck.CARD_VALUES[card[0]]
    
    def dealer_action(self):
        """Hit the dealer until 17 min (variable adjusts soft 17 stands) recursive"""
        hand = self.dealer_hand
        total, soft = self.calculate_hand_value(hand)
        
        if total < 17:
            self.hit(hand)
            self.dealer_action()
        if total == 17 and soft and self.HIT_SOFT_17:
            self.hit(hand)
            self.dealer_action()
            
    def check_win(self, player_hand, dealer_hand, player_blackjack=False, dealer_blackjack=False):
        """Check to see if player wins, loses, or pushes"""
        player_total, _ = self.calculate_hand_value(player_hand)
        dealer_total, _ = self.calculate_hand_value(dealer_hand)
        player_bust = self.check_bust(player_hand)
        dealer_bust = self.check_bust(dealer_hand)
        
        # Return player reslut and dealer blackjack
        if player_blackjack and not dealer_blackjack:
            return 'BJ', False
        elif player_blackjack and dealer_blackjack:
            return 'P', False
        elif dealer_blackjack:
            return 'L', False
        
        # Return player result and opposite outcome bust
        if (player_total > dealer_total or dealer_bust) and not player_bust:
            return 'W', dealer_bust
        elif (player_total < dealer_total and not dealer_bust) or (player_bust):
            return 'L', player_bust
        else:
            return 'P', player_bust
    
    def check_blackjack(self, hand, split_counter):
        """Returns true if hand is a black jack (cannor attain on splits)"""
        cards = len(hand)
        total, soft = self.calculate_hand_value(hand)
        return True if (cards==2 and total==21 and split_counter==0) else False
          
    def show_player_hand(self, player, i, hand):
        """Display players hand"""
        total, soft = self.calculate_hand_value(hand)
        print()
        print(f"Player {player}'s Hand {i}: {hand} | Value: {'Soft ' if soft else ''}{total}")
        
    def show_dealer_hand(self):
        """Display players hand"""
        total, soft = self.calculate_hand_value(self.dealer_hand)
        print()
        print(f"Dealer has {self.dealer_hand} | Value: {'Soft ' if soft else ''}{total}") 
        
    def show_table(self):
        """Display player hands and totals and dealers first card"""
        
        for player, hands in self.players_hands.items():
            for i, hand in enumerate(hands, start=1):
                total, soft = self.calculate_hand_value(hand)
                print(f"Player {player}'s Hand {i}: {hand} | Value: {'Soft ' if soft else ''}{total}")
        
        print()
        print('Dealer Shows:')
        print(f'{self.dealer_hand[0]} | Value: {self.get_value(self.dealer_hand[0])}')
        print()
        
    def reset_table(self,num_players):
        """Reset players and dealers hands to empty"""
        self.players_hands = {i :[[]] for i in range(1, num_players+1)} # set players with empty hands (double indexed for multiple hands)
        self.dealer_hand = []
        
    def load_new_shoe(self):
        self.deck = Deck(self.num_decks)
        
    def check_deck(self):
        """Get the count of remaining cards in the deck"""
        return self.deck.check_deck()

In [92]:
set((6,7,8)) == set((8,7,6))

True

# Create Game Function

In [5]:
def PlayBlackjackGame(num_players=1, num_decks=1, HIT_SOFT_17 = False, game_type='Shoe', cut=.8, rigged_deck=False, v1=11, v2=10, dv=6):
    
    # Create an instance of the bblackjack game
    game = Blackjack(num_players=num_players,num_decks=num_decks, HIT_SOFT_17 = HIT_SOFT_17, cut=cut)
    
    # Insert cut card to know when to reshuffle the deck before it runs out
    # If auto shuffle, then not needed as cards get shuffled after every round
    game.insert_cut()
    
    while True:
        # Get number of cards left in deck
        cards_left = game.deck.check_deck()
        print('NEW DEAL')
        print(f'{cards_left} Cards Remaining.')
        
        
        # Deal random or rigged
        if rigged_deck:
            game.rigged_deal(v1,v2,dv)
        else:
            game.initial_deal()

        # Show table to start
        game.show_table()

        # Check for player blackjacks {player:blackjack}
        blackjacks = {}
        for player in game.players_hands.keys():
            total, _ = game.calculate_hand_value(game.players_hands[player][0])
            blackjacks[player] = True if total == 21 else False


        # Check for dealer blackjack if Ace is showing and offer insurance
        dealer_blackjack = False
        if game.get_rank(game.dealer_hand[0]) == 'A':
            print('Insurance for half bet value? [y/n]')
            insurance = {}
            for player in game.players_hands.keys():
                 insurance[player] = input(f'Player {player}: ').strip().lower()

            # Check if dealers second cards value is 10
            if game.get_value(game.dealer_hand[1]) == 10:
                dealer_blackjack = True
                game.show_dealer_hand()
                print('Dealer Black Jack :( (insurance wins)')
            else:
                print('No Dealer Blackjack :) continue play (insurance lost).')

        # Check for dealer blackjack on 10 showing (will still count but there is no insurance offer and game play continues)
        dealer_blackjack_from_10 = True if game.get_value(game.dealer_hand[1]) == 10 and game.dealer_hand[1] == 'A' else False


        # Start Gameplay Loop (if dealer did not have a blackjack)
        if not dealer_blackjack:
            for player, hands in game.players_hands.items():
                print(f"\nPlayer {player}'s Turn.")
                split_counter = 0
                aces_split = False
                
                # Skip hand if player has blackjack
                if blackjacks[player]:
                    continue
            
                for i, hand in enumerate(hands, start=1):
                    # Auto hit hand if there is only one card (after split)
                    if len(hand) == 1:
                        game.hit(hand)

                    while True:
                        game.show_player_hand(player, i, hand)
                        action = game.get_player_action(hand, split_counter, aces_split)      
                        if action == 's':
                            print('Stand!')
                            break
                        elif action == 'h':
                            print('Hit!')
                            game.hit(hand)
                        elif action == 'p':
                            print('Split!')
                            # Check if split hand is aces
                            if game.get_rank(hand[0]) == 'A':
                                aces_split = True
                            game.split(hands, hand)
                            game.hit(hand)
                            split_counter += 1
                            if split_counter == game.MAX_SPLITS: print('Max Splits Reached.')
                        elif action == 'd':
                            print('Double!')
                            game.hit(hand)
                            game.show_player_hand(player, i, hand)
                            # chips.double_down()
                            break
                        elif action == 'q':
                            print('Force Quit')
                            sys.exit()
                        else:
                            print('Not sure how you got here')

                        if game.check_bust(hand):
                            game.show_player_hand(player, i, hand)
                            print('Busted!')
                            break

            # Make dealer actions
            game.dealer_action()
            print()
            print('---RESULTS---')
            game.show_dealer_hand()

            # Check hand wins
            for player, hands in game.players_hands.items():
                print(f"\nPlayer {player} results:")
                for i, hand in enumerate(hands, start=1):
                    game.show_player_hand(player, i, hand)
                    result, bust = game.check_win(hand, game.dealer_hand, blackjacks[player], dealer_blackjack_from_10)

                    if blackjacks[player] or dealer_blackjack_from_10:
                        print('Blackjack :) Win 3:2') if result == 'BJ' else print('Dealer Blackjack :(') if result == 'P' else print('Blackjack Tie!')  
                    elif result == 'W' and bust:
                        print('Player Win, Dealer Bust! Win 1:1')
                    elif result == 'W' and not bust:
                        print('Player Win, Player Beat Dealer! Win 1:1')
                    elif result == 'P':
                        print('Player Push, Player Tied Dealer!')
                    elif result == 'L' and bust:
                        print('Player Lost, Player Bust!')
                    elif result == 'L' and not bust:
                        print('Player Lost, Dealer Beat Player!')

        else:
            # Get player insurance outcomes
            print(f"\nPlayer {player} results:")
            for player in game.players_hands.keys():
                # payout insurance
                if insurance[player] == 'y':
                    print(f'Player {player} took insurance: Win 2:1')
                else:
                    print(f'Player {player} did not take insurance, lose.')
    
        # Clear table after deal
        game.reset_table(num_players)
        
        # Regenerate and shuffle deck if cut card was reached during the gameplay
        if game.cut_reached:
            print('Cut was reached during hand, new deck shuffling...')
            game.load_new_shoe()
            game.insert_cut()
            
        print()
                
            


In [284]:
PlayBlackjackGame(num_players=1, num_decks=1, rigged_deck=False, v1=10, v2=10, dv=11)

NEW DEAL
53 Cards Remaining.
Player 1's Hand 1: [('10', '♥'), ('3', '♣')] | Value: 13

Dealer Shows:
('7', '♦') | Value: 7


Player 1's Turn.

Player 1's Hand 1: [('10', '♥'), ('3', '♣')] | Value: 13


Chose: [H]it, [S]tand, [D]ouble Down:  s


Stand!
---RESULTS---

Dealer has [('7', '♦'), ('10', '♣')] | Value: 17

Player 1 results:

Player 1's Hand 1: [('10', '♥'), ('3', '♣')] | Value: 13
Player Lost, Dealer Beat Player!

NEW DEAL
49 Cards Remaining.
Player 1's Hand 1: [('K', '♠'), ('Q', '♠')] | Value: 20

Dealer Shows:
('8', '♠') | Value: 8


Player 1's Turn.

Player 1's Hand 1: [('K', '♠'), ('Q', '♠')] | Value: 20


Chose: [H]it, [S]tand, S[p]lit, [D]ouble Down:  s


Stand!
---RESULTS---

Dealer has [('8', '♠'), ('Q', '♦')] | Value: 18

Player 1 results:

Player 1's Hand 1: [('K', '♠'), ('Q', '♠')] | Value: 20
Player Win, Player Beat Dealer! Win 1:1

NEW DEAL
45 Cards Remaining.
Player 1's Hand 1: [('A', '♣'), ('K', '♥')] | Value: 21

Dealer Shows:
('6', '♠') | Value: 6


Player 1's Turn.

Player 1's Hand 1: [('A', '♣'), ('K', '♥')] | Value: 21


...

Chose: [H]it, [S]tand:  s


Stand!
---RESULTS---

Dealer has [('5', '♠'), ('A', '♦'), ('8', '♦'), ('6', '♥')] | Value: 20

Player 1 results:

Player 1's Hand 1: [('4', '♣'), ('7', '♥'), ('A', '♥'), ('A', '♠'), ('8', '♥')] | Value: 21
Player Win, Player Beat Dealer! Win 1:1

NEW DEAL
16 Cards Remaining.
Player 1's Hand 1: [('4', '♥'), ('8', '♣')] | Value: 12

Dealer Shows:
('5', '♣') | Value: 5


Player 1's Turn.

Player 1's Hand 1: [('4', '♥'), ('8', '♣')] | Value: 12


Chose: [H]it, [S]tand, [D]ouble Down:  s


Stand!
---RESULTS---

Dealer has [('5', '♣'), ('10', '♠'), ('10', '♦')] | Value: 25

Player 1 results:

Player 1's Hand 1: [('4', '♥'), ('8', '♣')] | Value: 12
Player Win, Dealer Bust! Win 1:1

NEW DEAL
11 Cards Remaining.


ValueError: too many values to unpack (expected 2)

In [83]:
# Initialize Blackjack game with 4 players and 6 decks
game = Blackjack(num_players=4, num_decks=6)
game.initial_deal()
game.show_table()

Player 1: [('K', 'Hearts'), ('8', 'Hearts')] | Value: 18
Player 2: [('8', 'Diamonds'), ('5', 'Diamonds')] | Value: 13
Player 3: [('3', 'Spades'), ('7', 'Spades')] | Value: 10
Player 4: [('7', 'Clubs'), ('10', 'Diamonds')] | Value: 17

Dealer Shows:
('5', 'Clubs')


# Monte Carlo Simulation

## Standard Hands (Hit / Stands)

In [26]:
%%time

# Initialize empty list of simulation data
sim_data = []

# Create parameters for simulations
dealer_upcards = np.arange(2, 12, 1)
dealt_totals = np.arange(20, 3, -1)
stand_limits = np.arange(12, 22, 1)

# Set amount of times to run each combination
runs = 1000

# Set game parameters
num_players=1
num_decks=6

# Create instance of game
simulation = Blackjack(num_players=num_players, num_decks=num_decks, HIT_SOFT_17 = False)

# Cycle through each possible up card
for dealer_upcard in dealer_upcards:
    print(f'Simulating for dealer showing: {dealer_upcard}')
    # Cycle through each possible dealt total
    for dealt_total in dealt_totals:
        # Get initial index for target stand list ie. if dealt total = 18, sim should start at standing at 18
        start_index = bisect.bisect_right(stand_limits, dealt_total) - 1
        for stand_limit in stand_limits[start_index:]:
            
            for run in range(1, runs+1):
                
                # Rig the deal that randomizes cards equal to desired total
                simulation.rigged_deal(total=dealt_total, upcard=dealer_upcard)
                action = 'Stand'
                
                # Run options for player (Hit / Stand)
                player_hand = simulation.players_hands[1][0]
                player_total = simulation.calculate_hand_value(player_hand)[0]
                
                while player_total < stand_limit:
                    action = 'Hit'
                    simulation.hit(player_hand)
                    player_hand = simulation.players_hands[1][0]
                    player_total = simulation.calculate_hand_value(player_hand)[0]
                    
                player_hand_size = len(player_hand)
                
                # Run dealer action
                simulation.dealer_action()
                dealer_hand = simulation.dealer_hand
                dealer_total = simulation.calculate_hand_value(dealer_hand)[0]
                dealer_hand_size = len(dealer_hand)
                
                result, bust = simulation.check_win(player_hand, dealer_hand)
                
                if result == 'W':
                    win, loss, push = 1, 0, 0
                    method = 'Dealer Bust' if bust else 'Player Total'
                elif result == 'L':
                    win, loss, push = 0, 1, 0
                    method = 'Player Bust' if bust else 'Dealer Total'
                else:
                    win, loss, push = 0, 0, 1
                    method='Push'
                    
                
                
                # Add data to simulation list
                sim_data.append([dealer_upcard, dealt_total, stand_limit, player_hand, player_total, player_hand_size, dealer_hand, dealer_total, dealer_hand_size, action, win, loss, push, method])
                
                # Reset table
                simulation.reset_table(num_players)
                
                # Auto shuffle deck
                simulation.auto_shuffle()


            
# Create dataframe and populate with outcome data
# simulation_results = pd.DataFrame(columns=['Dealer Upcard','Dealt Total','Stand Limit', 'Win','Bust','Push','Player Total','Dealer Total','Player Cards','Dealer Cards'])   
simulation_results = pd.DataFrame(sim_data, columns=['Dealer Upcard','Dealt Total','Stand Limit','Player Hand','Player Total','Player Hand Size','Dealer Hand', 'Dealer Total', 'Dealer Hand Size','Action','Win','Loss','Push','Method'])   
simulation_results

Simulating for dealer showing: 2
Simulating for dealer showing: 3
Simulating for dealer showing: 4
Simulating for dealer showing: 5
Simulating for dealer showing: 6
Simulating for dealer showing: 7
Simulating for dealer showing: 8
Simulating for dealer showing: 9
Simulating for dealer showing: 10
Simulating for dealer showing: 11
CPU times: total: 2min 16s
Wall time: 2min 17s


,Dealer Upcard,Dealt Total,Stand Limit,Player Hand,Player Total,Player Hand Size,Dealer Hand,Dealer Total,Dealer Hand Size,Action,Win,Loss,Push,Method
0,2,20,20,"[(Q, ♣), (K, ♣)]",20,2,"[(2, ♥), (J, ♥), (7, ♥)]",19,3,Stand,1,0,0,Player Total
1,2,20,20,"[(K, ♦), (10, ♥)]",20,2,"[(2, ♣), (4, ♦), (K, ♠), (Q, ♦)]",26,4,Stand,1,0,0,Dealer Bust
2,2,20,20,"[(10, ♣), (Q, ♦)]",20,2,"[(2, ♥), (5, ♦), (6, ♠), (7, ♠)]",20,4,Stand,0,0,1,Push
3,2,20,20,"[(Q, ♦), (K, ♥)]",20,2,"[(2, ♦), (2, ♠), (9, ♣), (4, ♣)]",17,4,Stand,1,0,0,Player Total
4,2,20,20,"[(J, ♣), (10, ♥)]",20,2,"[(2, ♣), (5, ♣), (2, ♥), (10, ♠)]",19,4,Stand,1,0,0,Player Total
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699995,11,4,21,"[(2, ♠), (2, ♣), (K, ♥), (5, ♣), (7, ♥)]",26,5,"[(A, ♥), (8, ♠)]",19,2,Hit,0,1,0,Player Bust
1699996,11,4,21,"[(2, ♠), (2, ♦), (3, ♠), (K, ♦), (5, ♠)]",22,5,"[(A, ♠), (6, ♣)]",17,2,Hit,0,1,0,Player Bust
1699997,11,4,21,"[(2, ♣), (2, ♣), (5, ♠), (2, ♣), (3, ♦), (4, ♣...",25,7,"[(A, ♦), (K, ♣)]",21,2,Hit,0,1,0,Player Bust
1699998,11,4,21,"[(2, ♣), (2, ♦), (5, ♠), (4, ♣), (5, ♣), (3, ♦)]",21,6,"[(A, ♣), (Q, ♣)]",21,2,Hit,0,0,1,Push


In [31]:
book = simulation_results.groupby(['Dealer Upcard','Dealt Total','Stand Limit','Action']).sum()[['Win','Loss','Push']].reset_index()
book['Total Games'] = book['Win'] + book['Push'] + book['Loss']
book['Win Rate'] = round(book['Win'] / book['Total Games'] * 100,2)
book['Push Rate'] = round(book['Push'] / book['Total Games'] * 100,2)
book['Loss Rate'] = round(book['Loss'] / book['Total Games'] * 100,2)
book['EVT'] = book['Win'] - book['Loss']
book['EVR'] = book['EVT'] / book['Total Games']
book

,Dealer Upcard,Dealt Total,Stand Limit,Action,Win,Loss,Push,Total Games,Win Rate,Push Rate,Loss Rate,EVT,EVR
0,2,4,4,Stand,326,674,0,1000,32.6,0.0,67.4,-348,-0.348
1,2,4,5,Hit,342,658,0,1000,34.2,0.0,65.8,-316,-0.316
2,2,4,6,Hit,344,656,0,1000,34.4,0.0,65.6,-312,-0.312
3,2,4,7,Hit,321,679,0,1000,32.1,0.0,67.9,-358,-0.358
4,2,4,8,Hit,359,635,6,1000,35.9,0.6,63.5,-276,-0.276
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695,11,19,19,Stand,391,478,131,1000,39.1,13.1,47.8,-87,-0.087
1696,11,19,20,Hit,83,884,33,1000,8.3,3.3,88.4,-801,-0.801
1697,11,19,21,Hit,47,925,28,1000,4.7,2.8,92.5,-878,-0.878
1698,11,20,20,Stand,518,349,133,1000,51.8,13.3,34.9,169,0.169


In [84]:
# Average EVR for stand limits against dealer up cards:
book.groupby(['Stand Limit']).mean()['EVR'].reset_index()

,Stand Limit,EVR
0,4,-0.389800
1,5,-0.402100
2,6,-0.396200
3,7,-0.391775
4,8,-0.371340
5,9,-0.330450
6,10,-0.276414
7,11,-0.192225
8,12,-0.117900
9,13,-0.134290


In [ ]:
# By the average EVR make book stand limits at least 12

In [77]:
book[(book['Dealer Upcard'] == 2) & (book['Dealt Total'] == 12)]

,Dealer Upcard,Dealt Total,Stand Limit,Action,Win,Loss,Push,Total Games,Win Rate,Push Rate,Loss Rate,EVT,EVR
116,2,12,12,Stand,348,652,0,1000,34.8,0.0,65.2,-304,-0.304
117,2,12,13,Hit,336,610,54,1000,33.6,5.4,61.0,-274,-0.274
118,2,12,14,Hit,359,588,53,1000,35.9,5.3,58.8,-229,-0.229
119,2,12,15,Hit,348,576,76,1000,34.8,7.6,57.6,-228,-0.228
120,2,12,16,Hit,346,583,71,1000,34.6,7.1,58.3,-237,-0.237
121,2,12,17,Hit,312,619,69,1000,31.2,6.9,61.9,-307,-0.307
122,2,12,18,Hit,311,638,51,1000,31.1,5.1,63.8,-327,-0.327
123,2,12,19,Hit,272,676,52,1000,27.2,5.2,67.6,-404,-0.404
124,2,12,20,Hit,217,744,39,1000,21.7,3.9,74.4,-527,-0.527
125,2,12,21,Hit,131,851,18,1000,13.1,1.8,85.1,-720,-0.720


,Stand Limit,EVR
0,4,-0.389800
1,5,-0.402100
2,6,-0.396200
3,7,-0.391775
4,8,-0.371340
5,9,-0.330450
6,10,-0.276414
7,11,-0.192225
8,12,-0.117900
9,13,-0.134290


In [71]:
# Get Max EVR for each Upcard / Dealt total Scenario
EVR_table = book.loc[book.groupby(['Dealer Upcard','Dealt Total'])['EVR'].idxmax()][['Dealer Upcard','Dealt Total','Stand Limit','Action','EVR']]

# Create Pivot, make dealers up card the columns
EVR_table.pivot(index=['Dealt Total', 'Action', 'Stand Limit'], columns='Dealer Upcard', values='EVR')
# EVR_table.groupby(['Dealt Total','Stand Limit','Action']).sum()

Dealer Upcard                      2      3      4      5      6      7   \
Dealt Total Action Stand Limit                                             
4           Hit    12             NaN -0.089 -0.017    NaN  0.008    NaN   
                   14          -0.077    NaN    NaN -0.007    NaN    NaN   
                   16             NaN    NaN    NaN    NaN    NaN    NaN   
                   17             NaN    NaN    NaN    NaN    NaN -0.094   
                   18             NaN    NaN    NaN    NaN    NaN    NaN   
...                               ...    ...    ...    ...    ...    ...   
17          Hit    19             NaN    NaN    NaN    NaN    NaN    NaN   
            Stand  17          -0.138 -0.066 -0.091 -0.048 -0.034 -0.058   
18          Stand  18           0.085  0.154  0.192  0.194  0.286  0.353   
19          Stand  19           0.372  0.392  0.403  0.423  0.513  0.625   
20          Stand  20           0.649  0.649  0.667  0.648  0.724  0.747   

Dealer Upcard                      8      9      10     11  
Dealt Total Action Stand Limit                              
4           Hit    12             NaN    NaN    NaN    NaN  
                   14             NaN    NaN    NaN    NaN  
                   16          -0.153    NaN    NaN    NaN  
                   17             NaN    NaN    NaN -0.425  
                   18             NaN -0.264 -0.318    NaN  
...                               ...    ...    ...    ...  
17          Hit    19             NaN    NaN    NaN -0.620  
            Stand  17          -0.364 -0.415 -0.486    NaN  
18          Stand  18           0.104 -0.154 -0.255 -0.332  
19          Stand  19           0.590  0.281 -0.071 -0.087  
20          Stand  20           0.789  0.792  0.445  0.169  

[73 rows x 10 columns]

In [70]:
EVR_table

,Dealer Upcard,Dealt Total,Stand Limit,Action,EVR
10,2,4,14,Hit,-0.077
25,2,5,12,Hit,-0.155
41,2,6,12,Hit,-0.172
60,2,7,16,Hit,-0.085
70,2,8,12,Hit,0.011
...,...,...,...,...,...
1682,11,16,18,Hit,-0.617
1688,11,17,19,Hit,-0.620
1691,11,18,18,Stand,-0.332
1695,11,19,19,Stand,-0.087


# Lucky Lucky Side Bets
* 19 or 20 = 2:1 -> Implied Odds 33%
* 21 = 3:1 -> Implied Odds 25%
* Suited 21 = 15:1 -> Implied Odds 6.25%
* 6 7 8 = 30:1 => Implied Odds 3.3%
* 7 7 7 = 50:1 -> Implied Odds 2%
* Suited 6 7 8 = 100:1 -> Implied Odds 1%
* Suited 7 7 7 = 200:1 -> Implied Odds 0.5%

In [ ]:
%%time

# Initialize empty list of simulation data
sim_data = []

# Create parameters for simulations
num_players = np.arange(1, 8, 1)
dealt_totals = np.arange(20, 3, -1)
stand_limits = np.arange(12, 22, 1)

# Set amount of times to run each combination
runs = 1000

# Set game parameters
num_players=1
num_decks=6

# Create instance of game
simulation = Blackjack(num_players=num_players, num_decks=num_decks, HIT_SOFT_17 = False)

# Book Template

In [245]:
d = {'Player':[20],'Dealer':[10], 'Action':['H'], 'DealerCards':[4],'W':[20],'P':[5],'L':[10]}
book = pd.DataFrame(d)
book['T'] = book['W'] + book['P'] + book['L']
#book['T2'] = book.iloc[:,4:7].sum(axis=1)
book['W Rate'] = round(book['W'] / book['T'] * 100,2)
book['P Rate'] = round(book['P'] / book['T'] * 100,2)
book['L Rate'] = round(book['L'] / book['T'] * 100,2)
book['EVT'] = book['W'] - book['L']
book['EVR'] = book['EVT'] / book['T']
book

,Player,Dealer,Action,DealerCards,W,P,L,T,W Rate,P Rate,L Rate,EVT,EVR
0,20,10,S,4,20,5,10,35,57.14,14.29,28.57,10,0.285714


In [241]:
book.iloc[:,4:7].sum(axis=1)

0    35
dtype: int64

In [199]:
# Define buttons
hit_button = widgets.Button(description="Hit")
stand_button = widgets.Button(description="Stand")
split_button = widgets.Button(description="Split")
double_button = widgets.Button(description="Double")

# Define function to execute on click
def on_hit_clicked():
    print("Hit!")
    
def on_stand_clicked():
    print("Stand!")

# Link button to function
hit_button.on_click(on_hit_clicked())
stand_button.on_click(on_stand_clicked())

# Display button
display(hit_button, stand_button)

Hit!
Stand!


Button(description='Hit', style=ButtonStyle())

Button(description='Stand', style=ButtonStyle())